In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 59.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 88.0 MB/s eta 0:00:00:00:01
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.4.0
    Uninstalling safetensors-0.4.0:
      Successfully uninstalled safetensors-0.4.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
  Attempting uninstall: transformers
    Found existing in

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    "INSAIT-Institute/BgGPT-7B-Instruct-v0.2",
    device_map='auto',
    torch_dtype=torch.bfloat16,
cache_dir='/notebooks/bggpt'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("INSAIT-Institute/BgGPT-7B-Instruct-v0.2")

In [14]:
tokenizer.tokenize("здравей свят")

['▁здраве', 'й', '▁свят']

In [12]:
tokens = tokenizer("Здравей свят")
tokens

{'input_ids': [1, 35834, 28819, 32645], 'attention_mask': [1, 1, 1, 1]}

In [15]:
len(tokenizer.vocab)

38000

In [10]:
{y: x for x,y in tokenizer.vocab.items() if y in  tokens['input_ids']}

{1: '<s>', 35834: '▁Здраве', 28819: 'й', 32645: '▁свят'}

In [11]:
len(tokenizer.vocab)

38000

In [26]:
sorted_vocab = sorted(tokenizer.vocab.items(), key=lambda x:x[1])

In [19]:
def to_cuda(toks):
    toks['input_ids'] = toks['input_ids'].cuda()
    toks['attention_mask'] = toks['attention_mask'].cuda()
    return toks

## Text Generation

In [20]:
toks = tokenizer("Васил Левски е български", return_tensors="pt",)
toks = to_cuda(toks)
res = model.generate(**toks, max_new_tokens=100)
res_text = tokenizer.batch_decode(res)[0]
res_text

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> Васил Левски е български революционер, идеолог и организатор на българската национална революция, национален герой. Основател е на Вътрешната революционна организация (ВРО) и на Българския революционен централен комитет (БРЦК). Известен е и като Апостола на свободата, заради организирането и разработването на революционна мрежа за освобождаване на България от османско владичество.\n\nПрез 1868 г. организира чета, която'

In [23]:
def generate(prompt):
    toks = tokenizer(prompt, return_tensors="pt",)
    toks = to_cuda(toks)
    res = model.generate(**toks, max_new_tokens=100)
    res_text = tokenizer.batch_decode(res)[0]
    return res_text


In [17]:
generate("Криско е")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> Криско е български рап изпълнител, автор на песни и музикален продуцент.\n\nБиография \nКристиан Талев, по-известен като Криско, е роден на 11 май 1988 г. в Габрово. Той е най-малкият от три деца. Майка му е от Габрово, а баща му от Плевен.\n\nЗапочва да се занимава с музика на 13-годишна възраст. Първоначално свири на'

In [18]:
generate("Спайдърмен спасява ")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> Спайдърмен спасява 1000 души в Маями\nСпайдърмен се появи в Маями, за да спаси 1000 души. Не, това не е началото на нов филм за супергероя, а част от рекламна кампания на застрахователната компания Geico.\nВъв видеото, качено в YouTube, Спайдърмен се появява на покрива на небостъргач в Маями и започва да спасява хора, вися'

## Instruction Tuning

In [24]:
generate("[INST] Кой е Васил Левски? [/INST]", )


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> [INST] Кой е Васил Левски? [/INST]Васил Левски, роден като Васил Иванов Кунчев на 18 юли 1837 г. в Карлово, е виден български революционер, национален герой, идеолог и организатор на българската национална революция. Известен е и с много псевдоними като Дякона, Апостола и др. Той основава Вътрешната революционна организация (ВРО), която по-късно се превръща в Българския революционен централен комитет'

In [28]:
generate("Кой е Васил Левски? ", )


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> Кой е Васил Левски? 1837-1873\nВасил Левски е роден на 18 юли 1837 г. в Карлово. Той е петото дете на Иван и Гина Кунчеви. Неговите братя са Христо, Петър и Яко.\n\nВасил Левски е роден на 18 юли 1837 г. в Карлово. Той е петото дете на Иван и Гина Кунчеви. Неговите братя'

## Hallucination

In [32]:
text = generate("[INST] Напиши текста на химна на България [/INST]")

print(text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Напиши текста на химна на България [/INST]Текстът на химна на България е:

Мила Родино, ти си земен рай,
твойта хубост, твоята прелест,
аз ги нося завинаги.

В сърцето ми винаги ще живееш,
ти си част от мен, аз съм част от теб,
завинаги.

Да, завинаги!</s>


## Inspecting model output

In [29]:
res = model(**toks, output_hidden_states= True)

`res.logits` - holds the predictions for each position. It is a probability distribution over the entire vocabulary. Normally, we are interested in the predictions for the last token as it will produce the next token.

In [30]:
res.logits.shape

torch.Size([1, 5, 38000])

In [34]:
import torch.nn.functional as F

In [38]:
F.softmax(res.logits[0][-1]).argmax()

/tmp/ipykernel_42/721971541.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(res.logits[0][-1]).argmax()


tensor(34861, device='cuda:0')

In [44]:
inv_vocab = sorted({y:x for x,y in tokenizer.vocab.items()})

In [47]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(38000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

`res.hidden_states` - holds the hidden states for each layer and each position. In our case we have one Embedding layer followed by 32 MistralDecoderLayer, so we get a total of 33 tensors of hidden_states.

In [46]:
len(res.hidden_states), res.hidden_states[-1].shape

(33, torch.Size([1, 5, 4096]))

In [47]:
def get_hs(model, toks):
    res = model(toks['input_ids'], output_hidden_states=True )
    hidden_states = res[2]
    last_hidden_state = hidden_states[-1]
    return last_hidden_state

In [48]:
last_hidden_state = get_hs(model, toks)

In [49]:
import pandas as pd

In [50]:
tokens = tokenizer.convert_ids_to_tokens(toks['input_ids'][0])
token_last_hidden_states = last_hidden_state[0].detach().cpu().float().numpy()
pd.DataFrame({'token': tokens, 'hs': token_last_hidden_states.tolist()})

token                                                 hs
0         <s>  [0.2265625, 0.205078125, 0.1884765625, -0.7539...
1      ▁Васил  [8.5, 7.8125, -3.703125, -4.90625, 1.4375, -4....
2     ▁Левски  [4.96875, 0.890625, -6.375, 3.203125, -2.98437...
3          ▁е  [10.0625, 5.84375, -3.1875, -0.5390625, -0.091...
4  ▁български  [-4.5625, 0.1630859375, 5.8125, -2.578125, -0....

In [ ]:
res.hidden_states